In [0]:
from pyspark.sql import functions as F

In [0]:
raw_kafka = (spark.readStream
  .format('kafka')
  .options(**kafka_opts)
  .load())

In [0]:
bronze_base = 'dbfs:/pipelines/orders/bronze'
ckpt_bronze = 'dbfs:/pipelines/orders/_ckpt_bronze'

In [0]:
# Persist raw bytes and metadata (don’t parse yet)
bronze_df = (raw_kafka
  .select(
      F.col('key').alias('kafka_key'),
      F.col('value').alias('value_bytes'),
      F.col('headers'),
      F.col('timestamp').alias('kafka_ts'),
      F.col('topic'), F.col('partition'), F.col('offset')
  ))



In [0]:
(bronze_df.writeStream
  .format('delta')
  .outputMode('append')
  .option('checkpointLocation', ckpt_bronze)
  .trigger(processingTime='5 seconds')
  .start(bronze_base))